In [ ]:


import os
import json
import requests
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from openai import OpenAI
import gspread
from oauth2client.service_account import ServiceAccountCredentials

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

class InterparkTicketCrawler:
    def __init__(self, creds='google.json', sheet_name='감사한 티켓팅 신청서'):
        scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
        creds = ServiceAccountCredentials.from_json_keyfile_name(creds, scope)
        self.sheet = gspread.authorize(creds).open(sheet_name).worksheet('Hot')

        # 캐시 파일 경로
        self.artist_cache_path = Path('artist_cache.json')
        self.hashtag_cache_path = Path('hashtag_cache.json')

        # 캐시 로딩
        self.artist_cache = self.load_cache(self.artist_cache_path)
        self.hashtag_cache = self.load_cache(self.hashtag_cache_path)

    def load_cache(self, path: Path) -> dict:
        if path.exists():
            try:
                with open(path, 'r', encoding='utf-8') as f:
                    return json.load(f)
            except:
                pass
        return {}

    def save_cache(self, cache: dict, path: Path):
        with open(path, 'w', encoding='utf-8') as f:
            json.dump(cache, f, ensure_ascii=False, indent=2)

    def fetch_data(self):
        url = "https://tickets.interpark.com/contents/api/open-notice/notice-list"
        params = {"goodsGenre": "ALL", "goodsRegion": "ALL", "offset": 0, "pageSize": 400, "sorting": "OPEN_ASC"}
        headers = {
            "user-agent": "Mozilla/5.0",
            "referer": "https://tickets.interpark.com/contents/notice"
        }
        r = requests.get(url, params=params, headers=headers)
        r.raise_for_status()
        return r.json()

    def filter_hot(self, data):
        hot = []
        for d in data:
            if d.get('goodsGenreStr') == '뮤지컬' and not d.get('isHot'):
                continue
            if d.get('goodsGenreStr') != '뮤지컬' and d.get('viewCount', 0) <= 1000:
                continue
            hot.append({
                '오픈시간': d.get('openDateStr', ''),
                '조회수': d.get('viewCount', 0),
                '예매타입': d.get('openTypeStr', ''),
                '제목': d.get('title', ''),
                '예매코드': d.get('goodsCode', ''),
                '장르': d.get('goodsGenreStr', ''),
                'Image': d.get('posterImageUrl', '')
            })
        return hot

    def extract_artist(self, title: str) -> str:
        if title in self.artist_cache:
            return self.artist_cache[title]

        prompt = f"""
아래는 콘서트 제목이야. 여기서 가수명이나 그룹명만 간단히 추출해줘. 뮤지컬일 경우 뮤지컬 제목만 추출해줘.**영문일 경우 한글도 같이 작성해야되고, 약어가 있으면 풀네임이랑 약어도 같이 작성해야해**
예시: 악동뮤지션 (악뮤, AKMU)
제목: {title}
가수명 or 뮤지컬 제목:"""

        try:
            res = client.chat.completions.create(
                model="gpt-4o",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.2,
            )
            artist = res.choices[0].message.content.strip().strip('"')
            self.artist_cache[title] = artist
            return artist
        except Exception as e:
            print(f"❌ OpenAI 오류 (가수명): {e}")
            return "불명"

    def generate_hashtags(self, title: str, artist: str, genre: str) -> str:
        key = f"{title}"
        if key in self.hashtag_cache:
            return self.hashtag_cache[key]

        prompt = f"""
콘서트 제목: {title}
가수 또는 뮤지컬 제목: {artist}
장르: {genre}

위 콘서트를 대리티켓팅 목적으로 트위터에 해시태그 10개를 한국어로 작성해줘.
형식: #블랙핑크콘서트 #블랙핑크 #BLACKPINK #블핑댈티 #대리티켓팅
조건: '#' 포함하고 띄어쓰기 없이, 한 줄로 콤마 없이 출력해줘.
"""

        try:
            res = client.chat.completions.create(
                model="gpt-4o",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.5,
            )
            hashtags = res.choices[0].message.content.strip()
            self.hashtag_cache[key] = hashtags
            return hashtags
        except Exception as e:
            print(f"❌ OpenAI 오류 (해시태그): {e}")
            return "#대리티켓팅"

    def add_ai_columns(self, df):
        print("🤖 가수명 + 해시태그 생성 중...")
        artists = []
        hashtags = []

        for _, row in tqdm(df.iterrows(), total=len(df)):
            title = row['제목']
            genre = row['장르']

            artist = self.extract_artist(title)
            hashtag = self.generate_hashtags(title, artist, genre)

            artists.append(artist)
            hashtags.append(hashtag)

        df['가수명'] = artists
        df['해시태그'] = hashtags

        self.save_cache(self.artist_cache, self.artist_cache_path)
        self.save_cache(self.hashtag_cache, self.hashtag_cache_path)

        return df

    def update_sheet(self, df):
        self.sheet.clear()
        if df.empty:
            print("📭 HOT 티켓 없음")
            return
        self.sheet.append_row(list(df.columns))
        for row in df.values.tolist():
            self.sheet.append_row(row)
        print(f"✅ {len(df)}개 티켓 업로드 완료")

    def run(self):
        raw = self.fetch_data()
        hot = self.filter_hot(raw)
        df = pd.DataFrame(hot)
        if df.empty:
            return df
        df = df.sort_values(by='오픈시간')
        df = self.add_ai_columns(df)
        self.update_sheet(df)
        return df

if __name__ == "__main__":
    df = InterparkTicketCrawler().run()
    if not df.empty:
        print("\n📋 HOT 티켓 요약:")
        print(df[['오픈시간', '제목', '가수명', '해시태그']].to_string(index=False))

🤖 가수명 + 해시태그 생성 중...


100%|██████████| 9/9 [00:20<00:00,  2.32s/it]


✅ 9개 티켓 업로드 완료

📋 HOT 티켓 요약:
               오픈시간                                                                    제목                                                                가수명                                                                                      해시태그
2025-07-14 20:00:00                                       2025 영탁 단독 콘서트 "TAK SHOW4" - 전주                                                                 영탁                      #영탁콘서트 #영탁 #TAKSHOW4 #전주콘서트 #영탁대리티켓팅 #대리티켓팅 #영탁팬 #콘서트티켓팅 #전주공연 #영탁공연
2025-07-15 20:00:00                                     2025 AKMU STANDING CONCERT ［악동들］                                                    악동뮤지션 (악뮤, AKMU)                   #악뮤콘서트 #악동뮤지션 #AKMU #악뮤 #악뮤대리티켓팅 #악뮤콘서트2025 #악동들콘서트 #대리티켓팅 #악뮤팬 #콘서트티켓팅
2025-07-15 20:00:00                                      뮤 내한공연 (MEW The Farewell Shows)                                                             뮤 (MEW)                          #뮤내한공연 #뮤콘서트 #뮤티켓팅 #뮤대리티켓팅 #뮤팬 #뮤음악 #콘서트대리 #티켓팅대

In [ ]:
# 랜덤 문구 생성
import json
import random
from datetime import datetime

def get_random_tweet():
    # JSON 파일 읽기
    with open('tweet_templates.json', 'r', encoding='utf-8') as f:
        templates = json.load(f)
    
    # 랜덤 선택
    template = random.choice(templates)
    
    # 시간 치환
    current_time = datetime.now().strftime("%H:%M")
    title = "세븐틴 댈티 티켓팅"
    content = template['content'].replace("{open_time}", current_time).replace("{title}", title)
    
    return content

# 사용 예시
if __name__ == "__main__":
    tweet = get_random_tweet()
    print(tweet)

세븐틴 댈티 티켓팅

⚠️ 이번에도 놓치시겠습니까?

📊 충격적 사실:
일반인 티켓팅 성공률 3%
우리 서비스 성공률 97%

💸 놓치면 손해보는 기회:
• VIP 정가 30만원 → 수고비 5만원
• 실패시 100% 환불 (타업체는 환불 X)

🕐 오픈시간: 11:33

친절한 상담: https://open.kakao.com/o/sAJ8m2Ah


In [ ]:
# 트위터 이미지랑 업로드하기
import os
import tweepy
import requests
from dotenv import load_dotenv

load_dotenv()

class SimpleTweet:
    def __init__(self):
        auth = tweepy.OAuthHandler(os.getenv('TWITTER_API_KEY'), os.getenv('TWITTER_API_SECRET'))
        auth.set_access_token(os.getenv('TWITTER_ACCESS_TOKEN'), os.getenv('TWITTER_ACCESS_TOKEN_SECRET'))
        self.api = tweepy.API(auth)
        self.client = tweepy.Client(
            consumer_key=os.getenv('TWITTER_API_KEY'),
            consumer_secret=os.getenv('TWITTER_API_SECRET'),
            access_token=os.getenv('TWITTER_ACCESS_TOKEN'),
            access_token_secret=os.getenv('TWITTER_ACCESS_TOKEN_SECRET')
        )

    def _download_image(self, url, path="temp.jpg"):
        r = requests.get(url, stream=True)
        if r.status_code == 200:
            with open(path, "wb") as f:
                for chunk in r.iter_content(1024):
                    f.write(chunk)
            return path
        return None

    def post(self, text, image_url=None):
        media_ids = []
        if image_url:
            path = self._download_image(image_url)
            if path:
                media = self.api.media_upload(path)
                media_ids.append(media.media_id)

        tweet = self.client.create_tweet(text=text, media_ids=media_ids if media_ids else None)
        print(f"https://twitter.com/gamsahanticket/status/{tweet.data['id']}")

def quick_post(title, open_time, image_url=None):
    text = f"""{title}

대리 티켓팅 진행
최근 세븐틴 / BTS / 블랙핑크 댈티 성공경력

선착순 할인 이벤트:
VIP 잡아도 수고비 5만원 선입금, 실패시 수고비 전액환불

🕐 오픈시간: {open_time}

https://open.kakao.com/o/sAJ8m2Ah

#티켓팅 #대리티켓팅 #콘서트 #선착순할인"""
    SimpleTweet().post(text, image_url)

if __name__ == "__main__":
    quick_post(
        "테스트 콘서트",
        "2025.02.15 (토) 20:00",
        "https://ticketimage.interpark.com/Play/image/large/25/25008966_p.gif"
    )

✅ 트위터 API 연결 성공
📝 트윗: 테스트 콘서트

대리 티켓팅 진행
최근 세븐틴 / BTS / 블랙핑크 댈티 성공경력

선착...
🖼️ 이미지: https://ticketimage.interpark.com/Play/image/large/25/25008966_p.gif
🌐 이미지 URL 다운로드 중: https://ticketimage.interpark.com/Play/image/large/25/25008966_p.gif
✅ 이미지 다운로드 완료: temp_image.jpg
📤 이미지 업로드 중: temp_image.jpg
✅ 이미지 업로드 성공! Media ID: 1943863892331180032
📝 트윗 게시 중...
✅ 이미지 트윗 게시 성공!
🔗 트윗 URL: https://twitter.com/gamsahanticket/status/1943863894411518290


In [8]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

def test_push_number_to_google_sheet():
    # 인증 범위 설정
    scope = [
        'https://spreadsheets.google.com/feeds',
        'https://www.googleapis.com/auth/drive'
    ]

    # 인증
    credentials = ServiceAccountCredentials.from_json_keyfile_name('google.json', scope)
    gc = gspread.authorize(credentials)

    # 시트 열기
    sheet = gc.open('감사한 티켓팅 신청서').worksheet('Hot')

    # 테스트 숫자 데이터
    value = 1234

    # 셀에 숫자 삽입 (USER_ENTERED 모드)
    sheet.update('B2', [[value]], value_input_option='USER_ENTERED')  # 조회수 열이라고 가정

    print("✅ 숫자 1234를 B2 셀에 입력 완료.")

if __name__ == '__main__':
    test_push_number_to_google_sheet()

=== 2025년 06월 15일 티켓 오픈 정보 ===
<b>🎫 2025년 06월 15일 티켓 오픈 정보 🎫</b>

<b>🟠 내일 [11:00]</b>
<b>뮤지컬 〈사랑의 하츄핑〉 - 서울앵콜 </b>
👁 조회수: 1109  |  🎟 예매코드: <code>25008505</code>  |  📌일반예매
───────────────────

<b>🎫 2025년 06월 15일 티켓 오픈 정보 🎫</b>

<b>🟠 내일 [11:00]</b>
<b>뮤지컬 〈사랑의 하츄핑〉 - 서울앵콜 </b>
👁 조회수: 1109  |  🎟 예매코드: <code>25008505</code>  |  📌일반예매
───────────────────
<b>🟠 내일 [13:00]</b>
<b>뮤지컬 〈사랑은 비를 타고〉 30주년 공연 </b>
👁 조회수: 1572  |  🎟 예매코드: <code>25004150</code>  |  📌마지막 티켓오픈
───────────────────

<b>🎫 2025년 06월 15일 티켓 오픈 정보 🎫</b>

<b>🟠 내일 [11:00]</b>
<b>뮤지컬 〈사랑의 하츄핑〉 - 서울앵콜 </b>
👁 조회수: 1109  |  🎟 예매코드: <code>25008505</code>  |  📌일반예매
───────────────────
<b>🟠 내일 [13:00]</b>
<b>뮤지컬 〈사랑은 비를 타고〉 30주년 공연 </b>
👁 조회수: 1572  |  🎟 예매코드: <code>25004150</code>  |  📌마지막 티켓오픈
───────────────────
<b>🟠 내일 [14:00]</b>
<b>뮤지컬 〈플레임즈〉 </b>
👁 조회수: 1372  |  🎟 예매코드: <code>25007077</code>  |  📌2차티켓오픈
───────────────────

<b>🎫 2025년 06월 15일 티켓 오픈 정보 🎫</b>

<b>🟠 내일 [11:00]</b>
<b>뮤지컬 〈사랑의 하츄핑〉 - 서울앵콜 </b>
👁 조회수: 1109  |  🎟 예매코

In [1]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

def test_push_number_to_google_sheet():
    # 인증 범위 설정
    scope = [
        'https://spreadsheets.google.com/feeds',
        'https://www.googleapis.com/auth/drive'
    ]

    # 인증
    credentials = ServiceAccountCredentials.from_json_keyfile_name('google.json', scope)
    gc = gspread.authorize(credentials)

    # 시트 열기
    sheet = gc.open('감사한 티켓팅 신청서').worksheet('Hot')

    # 테스트 숫자 데이터
    value = 1234

    # 셀에 숫자 삽입 (USER_ENTERED 모드)
    sheet.update('B2', [[value]], value_input_option='USER_ENTERED')  # 조회수 열이라고 가정

    print("✅ 숫자 1234를 B2 셀에 입력 완료.")

if __name__ == '__main__':
    test_push_number_to_google_sheet()

/var/folders/13/0c_y4nld6n3cjz79csr3tmhc0000gn/T/ipykernel_33243/4183897125.py:22: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet.update('B2', [[value]], value_input_option='USER_ENTERED')  # 조회수 열이라고 가정


✅ 숫자 1234를 B2 셀에 입력 완료.


In [3]:
import os
import requests
import json
import time
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from typing import List, Dict, Any, Set
import logging
from urllib.parse import urlparse
from pathlib import Path
import hashlib

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class InterparkTicketCrawler:
    """인터파크 HOT 티켓 크롤러 클래스"""
    
    def __init__(self, credentials_path: str = 'google.json', spreadsheet_name: str = '감사한 티켓팅 신청서', image_folder: str = 'Image'):
        self.credentials_path = credentials_path
        self.spreadsheet_name = spreadsheet_name
        self.image_folder = Path(image_folder)
        self.base_url = "https://tickets.interpark.com/contents/api/open-notice/notice-list"
        self.worksheet = None
        self.downloaded_urls: Set[str] = set()  # 이미 처리된 URL 추적
        self.downloaded_hashes: Set[str] = set()  # 이미 다운로드된 이미지 해시 추적
        self._setup_directories()
        self._setup_google_sheets()
        self._load_existing_images()
        
    def _setup_directories(self) -> None:
        """이미지 저장 폴더 생성"""
        try:
            self.image_folder.mkdir(exist_ok=True)
            logger.info(f"이미지 폴더 준비 완료: {self.image_folder}")
        except Exception as e:
            logger.error(f"이미지 폴더 생성 실패: {e}")
            raise
    
    def _load_existing_images(self) -> None:
        """기존 이미지 파일들의 해시값 로드"""
        try:
            for image_file in self.image_folder.glob('*'):
                if image_file.is_file() and image_file.suffix.lower() in ['.jpg', '.jpeg', '.png', '.gif', '.webp']:
                    try:
                        with open(image_file, 'rb') as f:
                            content = f.read()
                            image_hash = hashlib.md5(content).hexdigest()
                            self.downloaded_hashes.add(image_hash)
                    except Exception as e:
                        logger.warning(f"기존 이미지 해시 계산 실패 {image_file}: {e}")
            logger.info(f"기존 이미지 {len(self.downloaded_hashes)}개의 해시값 로드 완료")
        except Exception as e:
            logger.warning(f"기존 이미지 로드 실패: {e}")
    
    def _setup_google_sheets(self) -> None:
        """구글 시트 API 설정"""
        try:
            scope = [
                'https://spreadsheets.google.com/feeds',
                'https://www.googleapis.com/auth/drive'
            ]
            credentials = ServiceAccountCredentials.from_json_keyfile_name(
                self.credentials_path, scope
            )
            gc = gspread.authorize(credentials)
            self.worksheet = gc.open(self.spreadsheet_name).worksheet('Hot')
            logger.info("구글 시트 연결 성공")
        except Exception as e:
            logger.error(f"구글 시트 설정 실패: {e}")
            raise
    
    def _get_request_headers(self) -> Dict[str, str]:
        """API 요청 헤더 반환"""
        return {
            "host": "tickets.interpark.com",
            "sec-ch-ua-platform": "Windows", 
            "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36",
            "accept": "application/json, text/plain, */*",
            "sec-ch-ua": "\"Chromium\";v=\"134\", \"Not:A-Brand\";v=\"24\", \"Google Chrome\";v=\"134\"",
            "sec-ch-ua-mobile": "?0",
            "sec-fetch-site": "same-origin",
            "sec-fetch-mode": "cors", 
            "sec-fetch-dest": "empty",
            "referer": "https://tickets.interpark.com/contents/notice",
            "accept-encoding": "gzip, deflate, br, zstd",
            "accept-language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7"
        }
    
    def _get_request_params(self) -> Dict[str, Any]:
        """API 요청 파라미터 반환"""
        return {
            "goodsGenre": "ALL", 
            "goodsRegion": "ALL",
            "offset": 0,
            "pageSize": 400,
            "sorting": "OPEN_ASC"
        }
    
    def fetch_ticket_data(self) -> List[Dict[str, Any]]:
        """인터파크 API에서 티켓 데이터 가져오기"""
        try:
            response = requests.get(
                self.base_url, 
                params=self._get_request_params(), 
                headers=self._get_request_headers()
            )
            response.raise_for_status()
            logger.info("티켓 데이터 API 호출 성공")
            return response.json()
        except requests.RequestException as e:
            logger.error(f"API 호출 실패: {e}")
            raise
    
    def _get_image_extension(self, url: str) -> str:
        """URL에서 이미지 확장자 추출"""
        try:
            parsed_url = urlparse(url)
            path = parsed_url.path.lower()
            if path.endswith(('.jpg', '.jpeg')):
                return '.jpg'
            elif path.endswith('.png'):
                return '.png'
            elif path.endswith('.gif'):
                return '.gif'
            elif path.endswith('.webp'):
                return '.webp'
            else:
                return '.jpg'  # 기본값
        except:
            return '.jpg'
    
    def _generate_url_hash(self, url: str) -> str:
        """URL을 기반으로 고유한 해시 생성"""
        return hashlib.md5(url.encode('utf-8')).hexdigest()[:12]
    
    def _generate_filename(self, goods_code: str, image_url: str, title: str = "", index: int = 0) -> str:
        """파일명 생성 (중복 방지 개선)"""
        extension = self._get_image_extension(image_url)
        
        if goods_code and goods_code.strip():
            # 예매코드가 있는 경우
            safe_code = "".join(c for c in goods_code if c.isalnum() or c in ('-', '_'))
            return f"{safe_code}{extension}"
        else:
            # 예매코드가 없는 경우: URL 해시 + 제목 일부 사용
            url_hash = self._generate_url_hash(image_url)
            
            # 제목에서 안전한 문자만 추출 (최대 20자)
            safe_title = ""
            if title:
                safe_title = "".join(c for c in title if c.isalnum() or c in ('-', '_', ' '))
                safe_title = safe_title.replace(' ', '_')[:20]
            
            if safe_title:
                return f"ticket_{safe_title}_{url_hash}{extension}"
            else:
                return f"ticket_{url_hash}_{index:03d}{extension}"
    
    def _is_duplicate_image(self, image_content: bytes) -> bool:
        """이미지 내용이 중복인지 확인"""
        image_hash = hashlib.md5(image_content).hexdigest()
        if image_hash in self.downloaded_hashes:
            return True
        self.downloaded_hashes.add(image_hash)
        return False
    
    def download_image(self, image_url: str, goods_code: str, title: str = "", index: int = 0) -> str:
        """이미지 다운로드 (중복 방지 개선)"""
        if not image_url or not image_url.strip():
            logger.warning("이미지 URL이 없습니다.")
            return ""
        
        # URL 중복 체크
        if image_url in self.downloaded_urls:
            logger.info(f"이미 처리된 URL, 스킵: {image_url}")
            return ""
        
        filename = self._generate_filename(goods_code, image_url, title, index)
        file_path = self.image_folder / filename
        
        # 이미 파일이 존재하면 스킵
        if file_path.exists():
            logger.info(f"이미지 이미 존재, 스킵: {filename}")
            self.downloaded_urls.add(image_url)
            return str(file_path)
        
        try:
            # 이미지 다운로드
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
            }
            response = requests.get(image_url, headers=headers, timeout=30)
            response.raise_for_status()
            
            # 이미지 내용 중복 체크
            if self._is_duplicate_image(response.content):
                logger.info(f"중복 이미지 감지, 다운로드 스킵: {image_url}")
                self.downloaded_urls.add(image_url)
                return ""
            
            # 파일 저장
            with open(file_path, 'wb') as f:
                f.write(response.content)
            
            self.downloaded_urls.add(image_url)
            logger.info(f"이미지 다운로드 완료: {filename}")
            return str(file_path)
            
        except requests.RequestException as e:
            logger.error(f"이미지 다운로드 실패 ({image_url}): {e}")
            return ""
        except Exception as e:
            logger.error(f"파일 저장 실패 ({filename}): {e}")
            return ""
    
    def filter_hot_tickets(self, raw_data: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        """HOT 티켓만 필터링하여 필요한 정보 추출"""
        hot_tickets = []
        
        for index, ticket in enumerate(raw_data):
            if not ticket.get('isHot', False):
                continue
            
            goods_code = ticket.get('goodsCode', '')
            image_url = ticket.get('posterImageUrl', '')
            title = ticket.get('title', '')
            
            # 이미지 다운로드
            local_image_path = ""
            if image_url:
                local_image_path = self.download_image(image_url, goods_code, title, index)
                
            ticket_info = {
                '오픈시간': ticket.get('openDateStr', ''),
                '조회수': ticket.get('viewCount', 0),
                '예매타입': ticket.get('openTypeStr', ''),
                '제목': title,
                '예매코드': goods_code,
                '멀티오픈': ticket.get('hasMultipleOpenDates', False),
                '장르': ticket.get('goodsGenreStr', ''),
                '지역': ticket.get('goodsRegionStr', ''),
                '공연장': ticket.get('venueName', ''),
                'Image': image_url
            }
            hot_tickets.append(ticket_info)
        
        logger.info(f"HOT 티켓 {len(hot_tickets)}개 필터링 완료")
        return hot_tickets
    
    def run(self) -> pd.DataFrame:
        """크롤링 실행 메인 함수 - 결과는 데이터 프레임으로 반환"""
        logger.info("티켓 크롤링 시작")
        
        # 1. 데이터 가져오기
        raw_data = self.fetch_ticket_data()
        
        # 2. HOT 티켓 필터링
        hot_tickets = self.filter_hot_tickets(raw_data)
        
        # 3. 데이터프레임 생성
        df = pd.DataFrame(hot_tickets)
        
        if df.empty:
            logger.info("HOT 티켓이 없습니다.")
        else:
            logger.info(f"총 {len(df)}개의 HOT 티켓 발견")
        
        logger.info("티켓 크롤링 완료")
        return df

try:
    # 크롤러 인스턴스 생성
    crawler = InterparkTicketCrawler(image_folder='Image')
    
    # 크롤링 실행, 결과 데이터 프레임 받기
    result_df = crawler.run()
    
    # 결과 출력 (데이터 프레임만)
    if not result_df.empty:
        print("\n=== HOT 티켓 목록 (DataFrame) ===")
        print(result_df.to_string(index=False))
    else:
        print("현재 HOT 티켓이 없습니다.")
        
except Exception as e:
    logger.error(f"크롤링 실행 중 오류 발생: {e}")
    print(f"오류 발생: {e}")


2025-06-29 17:10:59,080 - INFO - 이미지 폴더 준비 완료: Image
2025-06-29 17:11:00,853 - INFO - 구글 시트 연결 성공
2025-06-29 17:11:00,880 - INFO - 기존 이미지 34개의 해시값 로드 완료
2025-06-29 17:11:00,896 - INFO - 티켓 크롤링 시작
2025-06-29 17:11:00,992 - INFO - 티켓 데이터 API 호출 성공
2025-06-29 17:11:00,993 - INFO - 이미지 이미 존재, 스킵: 25008963.gif
2025-06-29 17:11:00,993 - INFO - 이미지 이미 존재, 스킵: 25009182.gif
2025-06-29 17:11:00,993 - INFO - 이미지 이미 존재, 스킵: 25005777.gif
2025-06-29 17:11:00,994 - INFO - 이미지 이미 존재, 스킵: ticket_2025_KANGDANIEL_CONC_1f99bba846aa.jpg
2025-06-29 17:11:00,994 - INFO - 이미지 이미 존재, 스킵: 25008553.gif
2025-06-29 17:11:00,994 - INFO - 이미지 이미 존재, 스킵: 25009222.gif
2025-06-29 17:11:00,994 - INFO - 이미지 이미 존재, 스킵: ticket_2025_AKMU_STANDING_C_65358635e737.jpg
2025-06-29 17:11:00,995 - INFO - 이미지 이미 존재, 스킵: ticket_진격의_거인_오피셜_콘서트_-_Bey_712b3ff4ca19.jpg
2025-06-29 17:11:00,995 - INFO - 이미지 이미 존재, 스킵: ticket_HOSHI_X_WOOZI_FAN_CO_fb1747b4826e.jpg
2025-06-29 17:11:00,995 - INFO - 이미지 이미 존재, 스킵: ticket_TOMORROW_X_TOGETHER__6


=== HOT 티켓 목록 (DataFrame) ===
               오픈시간   조회수   예매타입                                                         제목     예매코드  멀티오픈      장르 지역             공연장                                                                               Image
2025-06-30 19:00:00  3100   일반예매                      2025 정세운 소극장 콘서트 〈Bittersweet〉 ENCORE 25008963  True     콘서트 서울          명화라이브홀                https://ticketimage.interpark.com/Play/image/large/25/25008963_p.gif
2025-06-30 19:00:00  3485   일반예매                                 2025 차우민 팬미팅 ［WRITTEN BY,］ 25009182 False     콘서트 서울           가빈아트홀                https://ticketimage.interpark.com/Play/image/large/25/25009182_p.gif
2025-07-01 14:00:00 11493 3차티켓오픈                       뮤지컬 〈위키드〉 내한 공연(WICKED The Musical)  25005777 False     뮤지컬 서울     블루스퀘어 신한카드홀                https://ticketimage.interpark.com/Play/image/large/25/25005777_p.gif
2025-07-01 20:00:00  5832   일반예매               2025 KANGDANIEL CONCERT ［ACT : NEW EPISODE］       

In [4]:
result_df.dtypes

오픈시간     object
조회수       int64
예매타입     object
제목       object
예매코드     object
멀티오픈       bool
장르       object
지역       object
공연장      object
Image    object
dtype: object